In [6]:
import sys
sys.path.append('/Users/abhishek/Projects/experimental-timeflow/')
import timeflow as tflow
import numpy as np

In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import tensorflow as tf
from tensorflow.python.ops import clip_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops.math_ops import sigmoid
from tensorflow.python.ops.math_ops import tanh
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.ops.rnn_cell import RNNCell, LSTMStateTuple
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.framework import dtypes
from tensorflow.python.ops import random_ops

In [4]:
def random_exp_initializer(minval=0, maxval=None, seed=None,
                               dtype=dtypes.float32):
    """Returns an initializer that generates tensors with an exponential distribution.

    Args:
      minval: A python scalar or a scalar tensor. Lower bound of the range
        of random values to generate.
      maxval: A python scalar or a scalar tensor. Upper bound of the range
        of random values to generate.  Defaults to 1 for float types.
      seed: A Python integer. Used to create random seeds. See
        [`set_random_seed`](../../api_docs/python/constant_op.md#set_random_seed)
        for behavior.
      dtype: The data type.

    Returns:
      An initializer that generates tensors with an exponential distribution.
    """

    def _initializer(shape, dtype=dtype, partition_info=None):
        return tf.exp(random_ops.random_uniform(shape, minval, maxval, dtype, seed=seed))

    return _initializer

In [ ]:
class PhasedLSTMCell():
    
    def __init__(self, inputs, num_units, alpha=0.001, r_on_init=0.05, tau_init=6.):
        self._num_units = num_units
        self.alpha = alpha
        self.r_on_init = r_on_init
        self.tau_init = tau_init
        
        
        #Building the graph
        dtype = input_layer.dtype
        #Initializing the kronos gate variables
        self.tau = vs.get_variable(
                "T", shape=[self._num_units],
                initializer=random_exp_initializer(0, self.tau_init), dtype=dtype)

        self.r_on = vs.get_variable(
            "R", shape=[self._num_units],
            initializer=init_ops.constant_initializer(self.r_on_init), dtype=dtype)

        self.s = vs.get_variable(
            "S", shape=[self._num_units],
            initializer=init_ops.random_uniform_initializer(0., self.tau.initialized_value()), dtype=dtype)

In [ ]:
class LSTMLayerBatch(object):

    def __init__(self, input_size, hidden_layer_size, input_placeholder):

        # Initialization of given values
        self.input_size = input_size
        self.hidden_layer_size = hidden_layer_size
        self.target_size = target_size

        # Weights and Bias for input and hidden tensor
        self.Wi = tf.Variable(tf.zeros(
            [self.input_size, self.hidden_layer_size]))
        self.Ui = tf.Variable(tf.zeros(
            [self.hidden_layer_size, self.hidden_layer_size]))
        self.bi = tf.Variable(tf.zeros([self.hidden_layer_size]))

        
        self.Wf = tf.Variable(tf.zeros(
            [self.input_size, self.hidden_layer_size]))
        self.Uf = tf.Variable(tf.zeros(
            [self.hidden_layer_size, self.hidden_layer_size]))
        self.bf = tf.Variable(tf.zeros([self.hidden_layer_size]))        
        
        
        self.Wog = tf.Variable(tf.zeros(
            [self.input_size, self.hidden_layer_size]))
        self.Uog = tf.Variable(tf.zeros(
            [self.hidden_layer_size, self.hidden_layer_size]))
        self.bog = tf.Variable(tf.zeros([self.hidden_layer_size]))        
        
        
        self.Wc = tf.Variable(tf.zeros(
            [self.input_size, self.hidden_layer_size]))
        self.Uc = tf.Variable(tf.zeros(
            [self.hidden_layer_size, self.hidden_layer_size]))
        self.bc = tf.Variable(tf.zeros([self.hidden_layer_size]))        
        
        # Placeholder for input vector with shape[batch, seq, embeddings]
        self._inputs = input_placeholder

        # Processing inputs to work with scan function
        self.processed_input = process_batch_input_for_RNN(self._inputs)
        
        self.initial_hidden = self._inputs[:, 0, :]
        self.initial_hidden= tf.matmul(
            self.initial_hidden, tf.zeros([input_size, hidden_layer_size]))
        
        
        self.initial_hidden=tf.pack([self.initial_hidden,self.initial_hidden])
        
    # Function for LSTM cell.
    def forward_step(self, previous_hidden_memory_tuple, x):
        """
        This function takes previous hidden state and memory tuple with input and
        outputs current hidden state.
        """
        
        previous_hidden_state,c_prev=tf.unpack(previous_hidden_memory_tuple)
        
        #Input Gate
        i= tf.sigmoid(
            tf.matmul(x,self.Wi)+tf.matmul(previous_hidden_state,self.Ui) + self.bi 
        )
        
        #Forget Gate
        f= tf.sigmoid(
            tf.matmul(x,self.Wf)+tf.matmul(previous_hidden_state,self.Uf) + self.bf 
        )
        
        #Output Gate
        o= tf.sigmoid(
            tf.matmul(x,self.Wog)+tf.matmul(previous_hidden_state,self.Uog) + self.bog
        )
        
        #New Memory Cell
        c_= tf.nn.tanh(
            tf.matmul(x,self.Wc)+tf.matmul(previous_hidden_state,self.Uc) + self.bc 
        ) 
        
        #Final Memory cell
        c= f*c_prev + i*c_
        
        #Current Hidden state
        current_hidden_state = o*tf.nn.tanh(c)


        return tf.pack([current_hidden_state,c])

    # Function for getting all hidden state.
    def get_outputs(self):
        """
        Iterates through time/ sequence to get all hidden state
        """

        # Getting all hidden state throuh time
        all_hidden_states = tf.scan(self.forward_step,
                                    self.processed_input,
                                    initializer=self.initial_hidden,
                                    name='states')
        all_hidden_states=all_hidden_states[:,0,:,:]
        
        return all_hidden_states

# Function to convert batch input data to use scan ops of tensorflow.
def process_batch_input_for_RNN(batch_input):
    """
    Process tensor of size [5,3,2] to [3,5,2]
    """
    batch_input_ = tf.transpose(batch_input, perm=[2, 0, 1])
    X = tf.transpose(batch_input_)

    return X

#### Synthetic data from NIPS2016

In [8]:
def gen_async_sin(async_sampling, resolution=None, batch_size=32, on_target_T=(5, 6), off_target_T=(1, 100),
                  max_len=125, min_len=85):
    half_batch = int(batch_size / 2)
    full_length = off_target_T[1] - on_target_T[1] + on_target_T[0] - off_target_T[0]
    
    # generate random periods
    posTs = np.random.uniform(on_target_T[0], on_target_T[1], half_batch)
    size_low = np.floor((on_target_T[0] - off_target_T[0]) * half_batch / full_length).astype('int32')
    size_high = np.ceil((off_target_T[1] - on_target_T[1]) * half_batch / full_length).astype('int32')
    low_vec = np.random.uniform(off_target_T[0], on_target_T[0], size_low)
    high_vec = np.random.uniform(on_target_T[1], off_target_T[1], size_high+1)
    
    negTs = np.hstack([low_vec,
                       high_vec])
    # generate random lengths
    if async_sampling:
        lens = np.random.uniform(min_len, max_len, batch_size)
    else:
        max_len *= int(1 / resolution)
        min_len *= int(1 / resolution)
        lens = np.random.uniform(min_len, max_len, batch_size)
    # generate random number of samples
    if async_sampling:
        samples = np.random.uniform(min_len, max_len, batch_size).astype('int32')
    else:
        samples = lens

    start_times = np.array([np.random.uniform(0, max_len - duration) for duration in lens])
    x = np.zeros((batch_size, max_len, 1))
    y = np.zeros((batch_size, 2))
    t = np.zeros((batch_size, max_len, 1))
    for i, s, l, n in zip(range(batch_size), start_times, lens, samples):
        if async_sampling:
            time_points = np.reshape(np.sort(np.random.uniform(s, s + l, n)), [-1, 1])
        else:
            time_points = np.reshape(np.arange(s, s + n * resolution, step=resolution), [-1, 1])
            
        if i < half_batch:  # positive
            _tmp_x = np.squeeze(np.sin(time_points * 2 * np.pi / posTs[i]))
            x[i, :len(_tmp_x), 0] = _tmp_x
            t[i, :len(_tmp_x), 0] = np.squeeze(time_points)
            y[i, 0] = 1.
        else:
            _tmp_x = np.squeeze(np.sin(time_points * 2 * np.pi / negTs[i - half_batch]))
            x[i, :len(_tmp_x), 0] = _tmp_x
            t[i, :len(_tmp_x), 0] = np.squeeze(time_points)
            y[i, 1] = 1.

    x = np.squeeze(np.stack([x, t], 2))

    return x, y, samples, posTs, negTs

In [18]:
x, y, samples, posTs, negTs = gen_async_sin(False, resolution=0.1, batch_size=2,
                                             off_target_T = [1, 100],
                                             on_target_T = [5, 6],
                                             min_len=50,max_len=125)

In [19]:
len(samples)

2

In [24]:
len(x[0,:,1])

1250

In [21]:
samples

array([ 1170.06170856,   603.65614742])